In [ ]:
import reflection_tasks
import example_cases
import app_constants
import json

### This is the main loop for the agentic reflection for NL to SQL generation.

In [ ]:
def run(reflection_state):

    iteration = 0
    while True:
        print(f"----Execution state at current stage:---------\n{json.dumps(reflection_state, indent=4)}")

        print("----Validation---------------------------------")
        # get validation result and analysis
        validation_result = reflection_tasks.validation(reflection_state)
        print(f"Validation result:\n{json.dumps(validation_result, indent=4)}")

        if validation_result['validation'] is True or \
                iteration >= app_constants.MAX_REFLECTION_ITERATIONS:
            print("End of iterations")
            break  # break out of the iteration loop
        else:
            print(f"Start iteration {iteration}")

        # prepare a prompt for reflection - 
        # using system prompt, user query, tables schema, 
        # tables rules, previously generated output
        print("----Prepare prompt for reflection--------------")
        reflection_prompt = reflection_tasks.generate_reflection_prompt(reflection_state)
        print("Prompt for performing reflection:")
        reflection_tasks.print_prompt(reflection_prompt)

        # LLM's response to the prompt
        print("----Invoke LLM to review and remedy------------")
        llm_response = reflection_tasks.llm_inference(reflection_prompt)

        # print thinking
        thinking = reflection_tasks.get_thinking_from_completion(llm_response)
        print(f"<thinking> tag in LLM's response:\n{thinking.strip()}")
        print()

        output_sql = reflection_tasks.get_sql_from_completion(llm_response)
        print(f"<sql> tag in LLM's response:\n{output_sql.strip()}")
        reflection_tasks.set_sql_in_genoutput(reflection_state, output_sql)

        iteration = iteration + 1
        reflection_tasks.set_iteration_in_genoutput(reflection_state, iteration)
        reflection_tasks.set_source_in_genoutput(reflection_state, 'Reflection-Pass')

    return reflection_state, validation_result


start = 0
end = 1
for example_case_index in range(start, end):
    reflection_state = example_cases.get_example_case(example_case_index)
    reflection_tasks.set_iteration_in_genoutput(reflection_state, 0)
    reflection_tasks.set_source_in_genoutput(reflection_state, 'First-Pass')

    generated, valresult = run(reflection_state)
    print("===========================================================\n\n")
    print(f"Validation: {valresult['validation']}\nSQL: {generated['sql']}")